In [ ]:
#Build an Autoencoder for efficient unsupervised feature learning.


In [1]:
import numpy as np
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras import regularizers

In [3]:
# Load the MNIST dataset
(x_train, _), (x_test, _) = mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 23s 2us/step


In [5]:
# Preprocessing: flatten the 28x28 images into vectors of size 784
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))


In [7]:
# Set the size of the encoded representation
encoding_dim = 32  # Compress the input to 32 dimensions

In [9]:
# Input placeholder
input_img = Input(shape=(784,))

In [11]:
# Encoding layers
encoded = Dense(128, activation='relu')(input_img)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu', activity_regularizer=regularizers.l1(10e-5))(encoded)

In [13]:
# Decoding layers
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(decoded)
decoded = Dense(784, activation='sigmoid')(decoded)

In [17]:
 #Autoencoder model: input -> encoded -> decoded
autoencoder = Model(input_img, decoded)

# Encoder model: input -> encoded (for feature extraction)
encoder = Model(input_img, encoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

In [19]:
# Train the autoencoder
autoencoder.fit(x_train, x_train,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(x_test, x_test))

# Encode some data for feature learning
encoded_imgs = encoder.predict(x_test)

print("Encoded feature representation shape:", encoded_imgs.shape)

Epoch 1/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - loss: 0.4811 - val_loss: 0.2631
Epoch 2/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.2631 - val_loss: 0.2628
Epoch 3/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - loss: 0.2632 - val_loss: 0.2627
Epoch 4/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.2631 - val_loss: 0.2629
Epoch 5/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.2631 - val_loss: 0.2626
Epoch 6/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.2627 - val_loss: 0.2627
Epoch 7/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 3s 12ms/step - loss: 0.2632 - val_loss: 0.2626
Epoch 8/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 0.2629 - val_loss: 0.2628
Epoch 9/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - loss: 0.2627 - val_loss: 0.2626
Epoch 10/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - loss: 0.2632 - val_loss: 0.2627
Epoch 11/50
235/235 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.2628 - val_loss: 0.2626
Epoch 12/50
235/235 ━━━━━━━━━━━━━━━━━━━━

In [21]:
from sklearn.metrics import mean_squared_error

# Predict the reconstructed images from the test data
reconstructed_imgs = autoencoder.predict(x_test)

# Calculate the Mean Squared Error (MSE) for each image
mse = np.mean(np.square(x_test - reconstructed_imgs), axis=1)

# Calculate the overall average MSE
average_mse = np.mean(mse)

print(f"Average MSE: {average_mse:.5f}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
Average MSE: 0.06750
